In [ ]:
from Bio import AlignIO
import os
from pathlib import Path
import blosum
from Bio.Align import substitution_matrices
from Bio.Align import PairwiseAligner
import numpy

In [ ]:
from matplotlib import pyplot
import matplotlib
%matplotlib inline
matplotlib.style.use("ggplot")
matplotlib.rcParams["figure.figsize"] = (30, 30)

In [ ]:
alignment_path = Path(".") / "Alignments"

In [ ]:
alignments = {x.split(".")[0]: AlignIO.read(alignment_path / x, format="clustal") for x in os.listdir("Alignments")}

In [ ]:
alignments

In [ ]:
def prepare_matrix(name):
    """
    Adapted substitution matrix:
    Maximal number was substracted from all the values, bringing them to negative values with maximum of 0.
    Then the values were reveresed so the least likely change is scored the highest.
    Finally, all the diagonal values (synonimous change) were changed to 0.
    Scaling everything (except indels) to get values between 0 and 1.
    Setting insertion to 2 and deletion to 3.
    Setting - => - to -1.
    """
    matrix = substitution_matrices.load(name)

    matrix = -(matrix - matrix.max())
    numpy.fill_diagonal(matrix, 0)
    # Temporary setting of the indels to 0
    matrix[-1,:] = 0
    matrix[:, -1] = 0
    # Scaling
    matrix = matrix / matrix.max()
    # Setting indels
    matrix[-1,:] = -0.5
    matrix[:, -1] = -1
    matrix[-1, -1] = -1.5
    return matrix

In [ ]:
comparisons = {}
alternative_alignments = {}
aligner = PairwiseAligner()
matrix = prepare_matrix("BLOSUM90")
aligner.substitution_matrix = substitution_matrices.load("BLOSUM90")

for key, alignment in alignments.items():
    species_raw = numpy.array(["_".join(seq.id.split("|")[1].split("_")[1:]) for seq in alignment])
    species_unique, counts = numpy.unique(species_raw, return_counts=True)

    if len(species_unique) > 1: # Only compare if there is a comparison to be made
        i_homo = int(numpy.where(species_raw == "Homo_sapiens")[0][0])
        reference_seq = alignment[i_homo]

        if (counts > 1).any(): # We have duplicates for the same species so we transfer all that are not maximally similar to the reference into the alternative alignment bucket
            to_remove = []
            alternative_alignments[key] = {}
            for i in numpy.where(counts > 1)[0]:
                i_s = [int(x) for x in numpy.where(species_raw == species_unique[i])[0]]
                aligner_results = numpy.array([aligner.score(str(reference_seq.seq).replace("-", "*"), str(alignment[i_current].seq).replace("-", "*")) for i_current in i_s])
                del i_s[aligner_results.argmax()]
                to_remove.extend(i_s)
                alternative_alignments[key][species_unique[i]] = {alignment[i_current].id: alignment[i_current].seq for i_current in i_s}
        
        species_raw = numpy.array(["_".join(seq.id.split("|")[1].split("_")[1:]) for seq in alignment])
        comparisons[key] = {}
        for i, (spec, seq) in enumerate(zip(species_raw, alignment)):
            if i != i_homo:
                comparisons[key][spec] = [matrix[x, y] for x, y in zip(str(reference_seq.seq).replace("-", "*"), str(seq.seq).replace("-", "*"))]

In [ ]:
# Cutoff AA for ECD vs the rest - unified across all the subunits through a comon alignment
ecd_cutoff = {"GABRA1": ("G", 251),
              "GABRA2": ("G", 251),
              "GABRA3": ("G", 276),
              "GABRA4": ("G", 257),
              "GABRA5": ("G", 258),
              "GABRA6": ("G", 241),
              "GABRB1": ("G", 244),
              "GABRB2": ("G", 243),
              "GABRB3": ("G", 244),
              "GABRD" : ("G", 248),
              "GABRE" : ("G", 278),
              "GABRG1": ("G", 271),
              "GABRG2L": ("G", 273),
              "GABRG3": ("G", 254),
              "GABRP" : ("L", 243),
              "GABRQ" : ("N", 266),
              "GABRR1": ("F", 282),
              "GABRR2": ("F", 262),
              "GABRR3": ("F", 268),}

# Transform the numbers into the correct ones for each alignment
def find_cutoff_number(sequence, aa, aa_number):
    j = 0
    for i, sequence_aa in enumerate(sequence):
        if sequence_aa != "-":
            j += 1
            if j == aa_number and sequence_aa == aa:
                return i

ecd_cutoff_ali = {}
for key, (aa, aa_number) in ecd_cutoff.items():
    for sequence in alignments[key]:
        if "Homo_sapiens" in sequence.id:
            break
    ecd_cutoff_ali[key] = find_cutoff_number(sequence, aa, aa_number)

In [ ]:
color_names = set(y for x in comparisons.values() for y in x.keys())
cmap = pyplot.get_cmap("tab10")
colors = {name: color for name, color in zip(color_names, numpy.apply_along_axis(matplotlib.colors.rgb2hex, 1, cmap(numpy.arange(len(color_names)))))}

comparisons_ecd =  {key: {subkey: comparisons[key][subkey][:i+1] for subkey in comparisons[key].keys()} for key, i in ecd_cutoff_ali.items()}
comparisons_rest = {key: {subkey: comparisons[key][subkey][i+1:] for subkey in comparisons[key].keys()} for key, i in ecd_cutoff_ali.items()}

print(len(comparisons["GABRA1"]["Pan_paniscus"]))
print(len(comparisons_ecd["GABRA1"]["Pan_paniscus"]))

In [ ]:
cytosines  = {"GABRA1": 166,
              "GABRA2": 166,
              "GABRA3": 191,
              "GABRA4": 172,
              "GABRA5": 173,
              "GABRA6": 156,
              "GABRB1": 161,
              "GABRB2": 160,
              "GABRB3": 161,
              "GABRD" : 164,
              "GABRE" : 195,
              "GABRG1": 188,
              "GABRG2L": 190,
              "GABRG3": 171,
              "GABRP" : 160,
              "GABRQ" : 183,
              "GABRR1": 198,
              "GABRR2": 178,
              "GABRR3": 184,}

# Converting to the alignment numbers
cytosines_adjusted = {}
for key, aa_number in cytosines.items():
    for sequence in alignments[key]:
        if "Homo_sapiens" in sequence.id:
            break
    cytosines_adjusted[key] = find_cutoff_number(sequence, "C", aa_number)


cytosine_length = max(cytosines_adjusted.values())
# max_length = max([len(y) for x in comparisons_ecd.values() for y in x.values()])
for key, values in comparisons_ecd.items():
    for subkey, subvalue in values.items():
        comparisons_ecd[key][subkey] = {"x": list(range(cytosine_length-cytosines_adjusted[key], len(subvalue)+cytosine_length-cytosines_adjusted[key])),
                                        "y": subvalue}

In [ ]:
def dict_to_plot(data, title="", ticks=True, colors=dict(), plot_gaps=False, vline=0, share_xaxis=True):
    """
    Function for plotting line-plots from a dictionary containing all the traces to be plotted.
    Argument ticks defines if the x-axis ticks should be plotted.
    Argument colors contains a dictionary where keys correspond to the keys in data and values being the hex codes for colors to use.
    Argument plot_gaps defines if the values where both sequences contain gaps should be plotted.
    Argument vline specifies the position of a vertical line if it should be plotted.
    """
    if share_xaxis:
        fig, axs = pyplot.subplots(len(data.keys()), 1, sharex="all")
    else:
        fig, axs = pyplot.subplots(len(data.keys()), 1, sharex="none")
    
    legends = {}

    for ax, (subtitle, subdata) in zip(axs, data.items()):
        ax.set_facecolor("white")
        for i, (key, data_series) in enumerate(subdata.items()):
            if plot_gaps:
                y = [x+(i*0.01) for x in data_series["y"]] # Adding a shift between the traces
            else:
                y = [x+(i*0.01) if x != -1.5 else i*0.01 for x in data_series["y"]] # Adding a shift between the traces and removing gaps
            if colors:
                legends[key] = ax.plot(data_series["x"], y, label=key, color=colors[key])[0]
            else:
                legends[key] = ax.plot(data_series["x"], y, label=key)[0]

        ax.hlines(-0.1, xmin=data_series["x"][0], xmax=data_series["x"][-1], colors="black", linestyles="solid")

        ax.set_title(subtitle)
        ax.grid(False)
        ax.spines["top"].set_visible(False)

        if plot_gaps:
            ax.set_yticks(ticks=(-1.5, -1, -0.5, 0, 1))
            ax.set_yticklabels(("Both gap", "Deletion", "Insertion", 0, 1))
        else:
            ax.set_yticks(ticks=(-1, -0.5, 0, 1))
            ax.set_yticklabels(("Deletion", "Insertion", 0, 1))

        ax.tick_params(
        axis="x",
        which="both",
        bottom=ticks,
        top=False,
        labelbottom=ticks
        )

    if vline:
        for i, ax in enumerate(axs):
            if i == 0:
                ax.axvline(x=vline, ymin=-1.3, ymax=1, c="black", linewidth=2, zorder=-1, clip_on=False, linestyle="-")
            else:
                ax.axvline(x=vline, ymin=0, ymax=2, c="black", linewidth=2, zorder=-1, clip_on=False, linestyle="-")

    leg = fig.legend(legends.values(), legends.keys(), ncol=len(legends.keys()), loc="upper center", bbox_to_anchor=(0.5, 0.1))

    fig.suptitle(title, fontsize=40)

In [ ]:
print("Cytosine: ", cytosines["GABRA1"])
print("End of ECD :", ecd_cutoff["GABRA1"])
print("Length :", len(comparisons_ecd["GABRA1"]["Pan_paniscus"]))
print()
print("Cytosine: ", cytosines["GABRA2"])
print("End of ECD :", ecd_cutoff["GABRA2"])
print("Length :", len(comparisons_ecd["GABRA2"]["Pan_paniscus"]))

In [ ]:
dict_to_plot(comparisons_ecd, ticks=False, colors=colors, title="ECD", plot_gaps=True, vline=cytosine_length)
# dict_to_plot(comparisons_rest, ticks=False, colors=colors, title="TMD&ICD", plot_gaps=True)

In [ ]:
print(alternative_alignments.keys())
[y.keys() for x in alternative_alignments.values() for y in x.values()]

In [ ]:
human_sequences = {}
for key, alignment in alignments.items():
    for sequence in alignment:
        if "Homo_sapiens" in sequence.id:
            human_sequences[key] = sequence
            break
human_sequences

In [ ]:
from  Bio import Align

In [ ]:
# Doing all the alternative alignments
alternative_alignments_results = {}
aligner = Align.PairwiseAligner()
for subalternative in alternative_alignments.values():
    for subsubalternative in subalternative.values():
        for alternative_key, alternative_sequence in subsubalternative.items():
            alignment_best = ("", "", 0)
            for key, sequence in human_sequences.items():
                alignment_test = aligner.align(str(sequence.seq).replace("-", ""), str(alternative_sequence).replace("-", ""))[0]
                if alignment_best[-1] < alignment_test.score:
                    alignment_best = (key, alignment_test, alignment_test.score)
            alternative_alignments_results[f"{alternative_key}_{alignment_best[0]}"] = alignment_best[1]
print(alternative_alignments_results)

In [ ]:
alternative_alignments_results.keys()

In [ ]:
alternative_alignments = {x.split(".")[0]: AlignIO.read(Path(".") / "Alternative_alignments" / x, format="clustal") for x in os.listdir("Alternative_alignments")}

comparisons_alternative = {}
for key, alignment in alternative_alignments.items():
    comparisons_alternative[key] = [matrix[x, y] for x, y in zip(str(alignment[0].seq).replace("-", "*"), str(alignment[1].seq).replace("-", "*"))]
    comparisons_alternative[key] = {"Comparison":{"x": list(range(len(comparisons_alternative[key]))),
                                                  "y": comparisons_alternative[key]}
                                    }

In [ ]:
dict_to_plot(comparisons_alternative, ticks=False, title="Alternative sequences", plot_gaps=False, share_xaxis=False)